<h1 id="tocheading">Finding Patterns in Data using IBM Power and PowerAI</h1>
**TODO** : Insert nice banner here
(techu / powerAI / sklearn / tensorflow /keras banner here)
<div id="toc"></div>

In this lab we will explore an open source data set, and discover how we can use the tools that are part of PowerAI to explore and discover patterns in the data.  For this lab, we will make use of the Lending Club data set.  Here is a brief description about Lending Club.


<img src="https://github.com/CatherineCao2016/pics/raw/master/lcintro.png" width="800" height="500" align="middle"/>

[Lending Club (LC)](https://www.lendingclub.com/) is the world’s largest online marketplace connecting borrowers and investors. It is transforming the banking system to make credit more affordable and investing more rewarding. Lending Club operates at a lower cost than traditional bank lending programs and pass the savings on to borrowers in the form of lower rates and to investors in the form of solid risk-adjusted returns.

The original data set is downloaded from [LC](https://www.lendingclub.com/info/download-data.action) covering complete loan data for all loans issued through the 2007-2018, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. Additional features include credit history, number of finance inquiries, address including zip codes, and state, and collections among others. It is quite rich and is really the only quality source of credit risk data we could locate; there is a large amount of value in the data and companies guard it relentlessly. Goldman Sachs’ new peer-to-peer lending platform called Marcus was built almost entirely using the Lending Club data.

**Important**

In this notebook, we will play with the LC data, conduct a set of exploratory analysis and try to apply various machine learning techniques to predict borrower’s default. We took a small sample of loans made in 2017 (150K) to help speed up the processing time for the lab


Note : to remove a lot of the busy verbose code, we are making using of a utility python file called lc_utils.py.  For implemenation details you can refer here  **TODO : link**

## Quick word on the data science method
<img src="https://github.com/dustinvanstee/random-public-files/raw/master/dsx-methodology.png" width="900" height="700" align="middle"/>

Here we will use these simple high level steps to work through a typical data science problem.  This workflow is meant to be a high level guide, but in practice this is a highly iterative approach ...


### Problem Understanding

**High level use case** - predict credit default analysis using lendingclub.com data

**TODO** - add some commentary here

**TODO** - link data dictionary, and add some commentary

### Goals

* Perform some initial analysis of the data for **Business Understanding**
* **Prepare the Data** for our visualization and modeling
* **Visualize** the data
* Model using **Dimension Reduction** and **Classification** techniques
* **Evaluate** the approach

## Data Understanding and Preparation
<img src="https://github.com/dustinvanstee/random-public-files/raw/master/data-preparation.png" width="800" height="500" align="middle"/>

In [1]:
# Environment bootstrapping
# !pip install jupyter-pip
# !pip3 install brunel
# !git fetch origin master
# !git reset --hard origin/master

In [6]:
%load_ext autoreload
%autoreload 2
import myenv as myenv
myenv.CLASS_ENVIRONMENT='dv-mac'
from lc_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CLASS_ENVIRONMENT = dv-mac


Using TensorFlow backend.


### Import Libraries

In [8]:
# Code functions that are needed to run this lab
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import time
from datetime import datetime
import math

import pandas as pd
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import glob

# custom library for some helper functions 
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import myenv as myenv
myenv.CLASS_ENVIRONMENT='dv-mac'
from lc_utils import *


### Load the Data
Here we load data that was previously downloaded from lendingclub.com.  For speed of this lab, we are restricting the number of loans ~ 180K

In [9]:
loan_df = load_sample_data()
loan_df.head()

**load_sample_data** : Loading /data/work/osa/2018-04-lendingclub/lending-club-loan-data/lendingclub.com/LoanStats_securev1_2016Q1.csv


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,76003542,NaN,16000.0,16000.0,16000.0,36 months,5.32%,481.84,A,A1,Security specialist,8 years,RENT,105000.0,Not Verified,Mar-2016,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,206xx,MD,15.02,1.0,Nov-2000,720.0,724.0,0.0,9.0,NaN,9.0,0.0,6219.0,27.9%,21.0,w,0.00,0.00,16098.340000,16098.34,16000.00,98.34,0.00,0.0,0.0,May-2016,16107.80,NaN,Nov-2016,679.0,675.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,23525.0,0.0,3.0,2.0,2.0,8.0,17306.0,69.0,0.0,1.0,3770.0,50.0,22300.0,1.0,3.0,1.0,3.0,3361.0,13632.0,29.7,0.0,0.0,124.0,184.0,22.0,8.0,0.0,52.0,NaN,8.0,9.0,0.0,2.0,3.0,2.0,5.0,7.0,6.0,14.0,3.0,9.0,0.0,0.0,0.0,2.0,95.2,0.0,0.0,0.0,47543.0,23525.0,19400.0,25243.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,76023477,NaN,17000.0,17000.0,17000.0,36 months,9.75%,546.55,B,B3,Kindergarten Teacher,< 1 year,MORTGAGE,40000.0,Verified,Mar-2016,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,440xx,NC,24.30,0.0,Aug-2006,705.0,709.0,0.0,NaN,NaN,4.0,0.0,14009.0,85.9%,8.0,w,1856.98,1856.98,16630.580000,16630.58,15143.02,1487.56,0.00,0.0,0.0,Nov-2017,546.55,Dec-2017,Nov-2017,799.0,795.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,31939.0,0.0,1.0,0.0,0.0,25.0,17930.0,64.0,0.0,0.0,4913.0,72.0,16300.0,1.0,0.0,1.0,0.0,7985.0,2291.0,85.9,0.0,0.0,25.0,115.0,35.0,25.0,0.0,35.0,NaN,6.0,NaN,0.0,3.0,3.0,3.0,5.0,1.0,3.0,7.0,3.0,4.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,44131.0,31939

### Descriptive Statistics (1D)
Lets look at some 1D and 2D descriptive statistics for this dataset

In this dataset, we have all types of data.  Numerical, Categorical, Ranked data.  This small module will take you through what is typical done to quickly understand the data



In [10]:
quick_overview(loan_df)

**quick_overview** : There are 133889 observations in the dataset.
**quick_overview** : There are 151 variables in the dataset.
**quick_overview** : 
Categorical vs Numerical
**quick_overview** : use df.dtypes ...
float64    112
object      39
Name: type, dtype: int64
**quick_overview** : 
******************Dataset Descriptive Statistics (numerical columns only) *****************************

 running df.describe() ....


,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,133887.000000,133887.000000,133887.000000,133887.000000,1.338870e+05,133870.000000,133887.000000,133887.000000,133887.000000,133886.000000,69322.000000,23671.000000,133887.000000,133887.000000,1.338870e+05,133887.000000,133887.000000,133887.000000,133887.000000,133887.000000,133887.000000,133887.000000,1.338870e+05,133887.000000,133887.000000,133887.000000,133887.000000,133887.000000,133887.000000,38074.000000,133887.0,2661.000000,2659.000000,133887.000000,133887.000000,1.338870e+05,133825.000000,133826.000000,133826.000000,133826.000000,130186.000000,133826.000000,116008.000000,133826.000000,133826.000000,133826.000000,133820.000000,1.338870e+05,133826.000000,133825.000000,133825.000000,133887.000000,133887.000000,132776.000000,132712.000000,133887.000000,133887.000000,130241.000000,133887.000000,133887.000000,133887.000000,133887.000000,132843.000000,33756.000000,120089.000000,47783.000000,133887.000000,133887.000000,133887.000000,133887.000000,133887.00000,133887.000000,133887.000000,133887.000000,133887.000000,133887.000000,126266.000000,133887.000000,133887.000000,133887.000000,133887.000000,132758.000000,133887.000000,133887.000000,1.338870e+05,1.338870e+05,1.338870e+05,1.338870e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,774.0,774.000000,774.0,774.000000,635.000000,774.000000,774.000000,1087.000000,1087.000000,1087.000000
mean,NaN,15589.394041,15589.394041,15581.984023,460.863923,8.046397e+04,19.547033,0.343476,695.750633,699.750827,0.576498,33.807810,66.361413,12.119922,0.238709,1.861627e+04,25.343245,5695.480540,5692.970475,10951.511813,10946.224173,8583.266797,2292.920647,1.102888e+00,74.221479,13.246068,3186.572260,687.161039,676.487710,0.021257,45.218154,1.0,108617.690135,18.603103,0.006969,259.246596,1.472398e+05,1.045373,2.797147,0.747157,1.625215,21.641175,35704.147819,70.741044,1.373769,2.930006,6183.582271,60.654499,3.636171e+04,0.952924,1.544121,2.187215,4.799159,13509.653738,10857.383932,59.762205,0.010120,21.343491,127.552652,188.682553,13.155243,7.799757,1.671402,23.845607,38.836799,6.812589,35.217672,0.518908,3.809533,5.917796,4.955836,8.00

Here we can get a quick assessment of the statistics for each column.  
**Quick Question** can you answer what was the average income for the 188K loan applicants ?

### Descriptive Statistics (2D)
Since we have 113 numerical variables, creating a 2D correlation plot may be time consuming and difficult to interpret

**TODO** write a nice util that shows just a couple vars of interest ....

### Create Loan Default column.  This is the column we will predict later
The **loan_status** column contains the information of whether or not the loan is in default. Here we will look at all the categorical values in loan_status, and create a new column based off that one.


In [11]:
loan_df = create_loan_default(loan_df)

**create_loan_default** : Unique values in loan_status
Current               79931
Fully Paid            35650
Charged Off           13043
Late (31-120 days)     3157
In Grace Period        1637
Late (16-30 days)       458
Default                  11
Name: loan_status, dtype: int64
**create_loan_default** : Dropping other values that are highly correlated with loan_status
**create_loan_default** : Dropping loan_status,total_rec_prncp,total_pymnt,total_pymnt_inv
**create_loan_default** : Unique values in default
0    117220
1     16669
Name: default, dtype: int64


### Handle Null Values ... Impute later for key columns ....

* handle null values (fill zero, impute mean/median/min/max/other, drop, etc)
* handle values that need to be re-cast (ie string to int, etc etc)


In [12]:
loan_df.describe()
columns_with_nans(loan_df)

**columns_with_nans** : id                                                 0
member_id                                     133889
loan_amnt                                          2
funded_amnt                                        2
funded_amnt_inv                                    2
term                                               2
int_rate                                           2
installment                                        2
grade                                              2
sub_grade                                          2
emp_title                                       8983
emp_length                                      8946
home_ownership                                     2
annual_inc                                         2
verification_status                                2
issue_d                                            2
pymnt_plan                                         2
url                                                2
desc                  


As you can see, we have some work to do to clean up the NaN values.  ... Luckily, we took care to process and clean this data below using a routine.  In practice, this is where data scientists spend a large portion of their time as this requires detailed domain knowledge to clean the data.  We have made a fair nubmer of assumptions about how to process the data which we won't go into due to time contraints for the lab.

In [13]:
#loan_df1 = drop_sparse_numeric_columns(loan_df)
#loan_df2 = drop_columns(loan_df1)
#loan_df3 = impute_columns(loan_df2)
#loan_df4 = handle_employee_length(loan_df3)
#loan_df5 = handle_revol_util(loan_df4)
#loan_df6 = drop_rows(loan_df5)

loan_df = clean_lendingclub_data(loan_df)


**clean_lendingclub_data** :  Running a couple routines to clean the data ...
**drop_sparse_numeric_columns** : Dropping columns with less than 0.025 pct cells populated
**transform** : Dropping member_id since its 0.0 pct populated
**transform** : Dropping annual_inc_joint since its 0.01987467230317651 pct populated
**transform** : Dropping dti_joint since its 0.019859734556236883 pct populated
**transform** : Dropping revol_bal_joint since its 0.0 pct populated
**transform** : Dropping sec_app_fico_range_low since its 0.0 pct populated
**transform** : Dropping sec_app_fico_range_high since its 0.0 pct populated
**transform** : Dropping sec_app_earliest_cr_line since its 0.0 pct populated
**transform** : Dropping sec_app_inq_last_6mths since its 0.0 pct populated
**transform** : Dropping sec_app_mort_acc since its 0.0 pct populated
**transform** : Dropping sec_app_open_acc since its 0.0 pct populated
**transform** : Dropping sec_app_revol_util since its 0.0 pct populated
**transform**

In [14]:
# Final Sanity check ....
columns_with_nans(loan_df)

**columns_with_nans** : id                            0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installment                   0
grade                         0
sub_grade                     0
home_ownership                0
annual_inc                    0
verification_status           0
issue_d                       0
pymnt_plan                    0
purpose                       0
zip_code                      0
addr_state                    0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
fico_range_low                0
fico_range_high               0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
total_acc                     0
initial_list_status           0
out_prncp                     0
out_prncp_inv                 0
total_rec_int   

### Data Preparation - Handle Time Objects
Sometimes for columns that contain date information, you may want to break them down into individual columns like month, day, day of week etc.  For our use case, we will create a new column called `time_history` that will indicate how long an applicant has been a borrower.  This is an example of **feature engineering**.  Essentially, using business logic to create a new column (feature) that may have predictive value.

In [15]:
loan_df = create_time_features(loan_df)

**create_time_features** : Creating new column called time_history : Calculated feature showing how long applicant has been a borrower..


### Convert Categorical Data to One hot encode ###
**TODO** explain here 

In [16]:
loan_df = one_hot_encode_keep_cols(loan_df)

**one_hot_encode_keep_cols** : Dropping these columns since they are greater than cardinality limit of 50
**one_hot_encode_keep_cols** : ['id', 'int_rate', 'zip_code', 'addr_state']
**one_hot_encode_keep_cols** : Keeping these cols
**one_hot_encode_keep_cols** : ['term', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'initial_list_status', 'application_type', 'disbursement_method', 'debt_settlement_flag', 'emp_bin']


### Final Result after data prep ....

In [17]:
loan_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,issue_d,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,acc_now_delinq,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mort_acc,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,total_bc_limit,default,revol_util_1,time_history,36 months,60 months,A,B,C,D,E,F,G,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,MORTGAGE,OWN,RENT,Not Verified,Source Verified,Verified,n,y,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,f,w,Individual,Joint App,Cash,DirectPay,N,Y,0_3yrs,4_6yrs,gt_6yrs,id
0,16000.0,16000.0,16000.0,481.84,105000.0,2016-03-01,15.02,1.0,2000-11-01,720.0,724.0,0.0,9.0,0.0,6219.0,21.0,0.00,0.00,98.34,0.00,0.0,0.0,16107.80,679.0,675.0,0.0,1.0,0.0,0.0,3.0,2.0,2.0,8.0,17306.0,69.0,0.0,1.0,3770.0,50.0,1.0,3.0,1.0,3.0,3361.0,13632.0,29.7,0.0,0.0,0.0,95.2,0.0,0.0,0.0,23525.0,19400.0,0,0.279896,5599,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,76003542
1,17000.0,17000.0,17000.0,546.55,40000.0,2016-03-01,24.30,0.0,2006-08-01,705.0,709.0,0.0,4.0,0.0,14009.0,8.0,1856.98,1856.98,1487.56,0.00,0.0,0.0,546.55,799.0,795.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,25.0,17930.0,64.0,0.0,0.0,4913.0,72.0,1.0,0.0,1.0,0.0,7985.0,2291.0,85.9,0.0,0.0,0.0,100.0,100.0,0.0,0.0,31939.0,16300.0,0,0.451772,3500,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,76023477
2,15000.0,15000.0,15000.0,359.30,70000.0,2016-03-01,9.99,0.0,2002-01-01,690.0,694.0,0.0,6.0,0.0,11630.0,22.0,0.00,0.00,2137.08,17.97,0.0,0.0,13106.47,754.0,750.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,12.0,15692.0,72.0,1.0,2.0,5840.0,60.0,3.0,0.0,6.0,3.0,4554.0,9808.0,38.7,0.0,0.0,3.0,95.5,33.3,0.0,0.0,27322.0,16000.0,0,0.436725,5173,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,75993535
3,2425.0,2425.0,2425.0,84.44,25000.0,2016-03-01,38.50,0.0,2007-09-01,705.0,709.0,0.0,7.0,0.0,10604.0,15.0,0.00,0.00,228.18,0.00,0.0,0.0,2062.67,714.0,710.0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,24.0,15963.0,65.0,0.0,0.0,7288.0,72.0,0.0,0.0,0.0,1.0,13860.0,12.0,99.8,0.0,0.0,2.0,100.0,100.0,0.0,0.0,26567.0,7300.0,0,0.813877,3104,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,73458582
4,29900.0,29900.0,29900.0,1007.31,68000.0,2016-03-01,17.49,0.0,1991-11-01,675.0,679.0,2.0,14.0,1.0,13984.0,39.0,17215.02,17215.02,4396.13,0.00,0.0,0.0,1007.31,604.0,600.0,1.0,1.0,0.0,2.0,2.0,1.0,2.0,3.0,10861.0,72.0,3.0,4.0,2917.0,66.0,0.0,0.0,5.0,6.0,40787.0,703.0,91.1,0.0,0.0,2.0,100.0,100.0,1.0,0.0,24845.0,7900.0,1,0.318658,8887,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,73511090


<img src="https://github.com/dustinvanstee/random-public-files/raw/master/data-visualization.png" width="800" height="500" align="middle"/>

### Data Visualization
As you saw, when you 'describe' a data frame, you get a table statistics showing you the mean,min,max and other statistics about each column.  This is good, but sometimes its also good to look at the histograms of the data as well.  Lets Visualize some of the distributions from our dataset


In [ ]:
plot_histograms(loan_df)

In [ ]:
### A word on visualization libararies.


# Brunel Example
## The Growth of Lending Club
### Here we use the builtin Brunel Visualization graphics package
Lending club has been expanding over the years in terms of total loan volume and average loan size.

In [ ]:
# Build a statistics data frame based on issue date
# aggregate on loan amount
# loan_stats = pd.concat([loan_df.groupby('issue_d').mean()['loan_amnt'].to_frame().rename(columns = {'loan_amnt':'loan_average'}), loan_df.groupby('issue_d')['loan_status'].count().to_frame().rename(columns = {'loan_status':'loan_count'})], axis=1)

In [ ]:
#%brunel data('loan_stats') line x(ISSUE_D) y(loan_average, loan_count) color(#series) tooltip(#all) :: width=900, height=350 

**TODO** Add more Brunel if i get it in nimbix

<img src="https://github.com/dustinvanstee/random-public-files/raw/master/modeling.png" width="800" height="500" align="middle"/>

### Train / Test set creation

In [ ]:
%load_ext autoreload
%autoreload 2
from lc_utils import *

In [ ]:
loan_df.head()
my_analysis = lendingclub_ml(loan_df)

In [ ]:
# Create a train / test split of your data set.  Paramter is test set size percentage 
# Returns data in the form of dataframes
my_analysis.create_train_test(test_size=0.4)

In [ ]:
### Correlation to defualt [REMOVE]
#my_analysis.train_df.dtypes
corr_vs_1var(my_analysis.train_df, 'default')
#my_analysis.train_df.head()
#my_analysis.train_df['default'].corr(my_analysis.train_df['TX'])

In [ ]:
my_analysis.X_train_scaled.head()

For this modeling exercise we will perform a couple of tasks, dimension reduction and classification as shown in the following diagram.

<img src="https://github.com/dustinvanstee/random-public-files/raw/master/techu-modeling-workflow.png" width="800" height="500" align="middle"/>

**Dimension Reduction** is useful in scenarios when you have a large number of columns and you would like to reduce that down to a compressed representation .  In this lab we will try 2 methods of dimension reduction.  It will be your choice to decide which method you want to use for the classification part of the lab ! (you could even decice to bypass this if you want ...)


## Dimension Reduction - PCA

<img src="https://github.com/dustinvanstee/random-public-files/raw/master/techu-pca.png"  width="200" height="125" align="middle"/>

Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. The number of principal components is less than or equal to the number of original variables.

In [ ]:
# dim red using PCA
my_analysis.build_pca_model(n_components=50)


## Dimension Reduction - AutoEncoder

<img src="https://github.com/dustinvanstee/random-public-files/raw/master/techu-ae.png"  width="600" height="375" align="middle"/>

**TODO : AE Writeup **

In [ ]:
my_analysis.build_ae_model(ae_layers=[100,25,6,25,100], regularization=0.001, epochs=1, folds=2, k_tries=1)

# Now update our test dataframe with new columns that are predicted by our PCA and Autoencoder models.  
Here we will now take the models that we built and pass our test data set through the models.   By doing this, we will have reduced the number features in our data set by a significant amount (177 => 5!)  .  

In this step we will add new columns to our test/train data frames for both our PCA model and our autoencoder model.  This is required for some followon visualization, and training steps ahead

In [ ]:
my_analysis.update_train_test_df()

In [ ]:
my_analysis.visualize_dimred_results(mode='pca')

In [ ]:
my_analysis.visualize_dimred_results(mode='ae')

In [ ]:
bob_heatmap_lc(my_analysis.test_df,sortColumn='PC0',add_corr=1)

# Final Step - Lending Club Default Prediction

<img src="https://github.com/dustinvanstee/random-public-files/raw/master/techu-modeling-traintest.png"  width="600" height="375" align="middle"/>

Here we will build a classifier to predict if loan will fail or not.  We will us a 
** Deep Learning Classifier **  .  You will have 3 options for data sources, 
* the raw data
* PCA dimension reduction features
* Autoencoder features

To evaluate our model, we will use a simple contingency table.  However, this is a fairly simplistic method.  Better method that can data scientists use are F1 score, and PR/ROC curves.

Step 1 here is to set our baseline result.  In this example, we are dealing with a **skewed** dataset.  This means, on average, most people will not default, and they pay their loan off.  If you built a classifier that just predicted no default, you would be right most of the time.  Lets see the stats from our dataset below....

In [ ]:
# Set our baseline
my_analysis.train_df['default'].describe()


As you can see, only 15.5% of the applicants default.  Any classifier we build must be better than this, or we aren't doing a very good job ;)

In [ ]:
mode = 'pca' # ae , all, 

if(mode == 'pca') :
            x_cols = [x for x in my_analysis.train_df.columns if 'PC' in x]
elif(mode == 'ae') :
            x_cols = [x for x in my_analysis.train_df.columns if 'AE' in x]
elif(mode == 'all') :
            x_cols = [x for x in my_analysis.train_df.columns if 'AE' in x]

my_analysis.build_evaluate_dl_classifier(x_cols, epochs=25)


In [ ]:
94.6% !
plot learning rate !!

### Credits (TODO)
* [Data Preparation](https://apsportal.ibm.com/analytics/notebooks/9ef75f73-140a-4618-9292-1de51f5f331c/view?projectid=399ab81a-5140-4d51-a5df-b6e82d51db85&context=analytics)
* [Data Visualization](https://apsportal.ibm.com/analytics/notebooks/24dd6830-8a01-4bde-b42d-1d040079af16/view?projectid=399ab81a-5140-4d51-a5df-b6e82d51db85&context=analytics)
* [Modelling/Eval/Deploy](https://apsportal.ibm.com/analytics/notebooks/2d84420b-e005-4b95-99f6-a56148305fbc/view?projectid=399ab81a-5140-4d51-a5df-b6e82d51db85&context=analytics)
* [Web App]()


